## Import libraries

In [2]:
import numpy as np
import librosa as lbr
import librosa.display as display
import wave
import math
import matplotlib.pyplot as plt

## Loop over folders in order to get sample rate and number of frames

In [3]:
folders = ["blues", "classical", "country", "disco", "hiphop", "jazz", "metal", "pop", "reggae", "rock"]

In [4]:
lsr = []
lnf = []

def par(wav):
    try:
        obj = wave.open(wav, 'rb')
        sr = obj.getframerate()
        lsr.append(sr)
        nframes = obj.getnframes()
        lnf.append(nframes)
        obj.close()
    except wave.Error as e:
        print(wav)

In [5]:
pre = "/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original"

for fol in folders:
    for i in range(100):
        if i < 10:
            path = f"{fol}/{fol}.0000{i}.wav"
        else:
            path = f"{fol}/{fol}.000{i}.wav"
        #print(path)
        par(path)

/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/jazz/jazz.00054.wav


In [6]:
unique_sr = list(set(lsr))
print(unique_sr)

unique_nf = list(set(lnf))
print(unique_nf)

[22050]
[22050]


### Il sample rate è uguale per tutti i file.
### Il numero di frames, invece, non è lo stesso, quindi controllo quale
### numero compare maggiormente.

In [7]:
from collections import Counter

frequency_dict = Counter(lnf)
print(frequency_dict)

Counter({22050: 999})


## Modifico la durata dei file con numero frames diverso da 661794

In [8]:
from pydub import AudioSegment
import os

def cambia_numero_di_frame(file_input):
    # Imposta il numero di frame desiderato
    numero_frame_desiderato = 661794

    # Crea la cartella "Audio modificati" nella directory corrente se non esiste già
    output_directory = 'Audio modificati'
    os.makedirs(output_directory, exist_ok=True)
    
    # Carica il file audio
    audio = AudioSegment.from_file(file_input)
    
    # Ottieni il sample rate dell'audio
    sample_rate = audio.frame_rate
    
    # Calcola il numero di frame attuale
    numero_frame_attuale = len(audio.get_array_of_samples())
    
    # Se il numero di frame è già quello desiderato, non fare nulla
    if numero_frame_attuale == numero_frame_desiderato:
        print(f"Il file {file_input} ha già {numero_frame_desiderato} frame.")
        return
    
    # Calcola il fattore di scalamento per ottenere il numero di frame desiderato
    fattore_di_scalamento = numero_frame_attuale / numero_frame_desiderato
    
    # Modifica la velocità dell'audio in base al fattore di ridimensionamento
    audio_modificato = audio._spawn(audio.raw_data, overrides={
        "frame_rate": int(audio.frame_rate / fattore_di_scalamento)
    }).set_frame_rate(sample_rate)  # Imposta di nuovo il frame rate originale
    
    # Salva il file modificato nella cartella "Audio modificati"
    output_path = os.path.join(output_directory, os.path.basename(file_input))
    audio_modificato.export(output_path, format="wav")
    
    print(f"Audio modificato e salvato in {output_path} con {numero_frame_desiderato} frame.")


In [9]:
# devo lanciare nuovamente quello che recupera nframes e cambiare quelli diversi da 661794

In [ ]:
for fol in folders:
    for i in range(100):
        if i < 10:
            path = f"{fol}/{fol}.0000{i}.wav"
        else:
            path = f"{fol}/{fol}.000{i}.wav"
        #print(path)
        try:
            cambia_numero_di_frame(path)
        except :
            print(path)

In [10]:
hop = 512
frame_size = 2048
sr = 22050
lsp = []

In [11]:
def spec(audio):
    return lbr.stft(audio, n_fft=frame_size, hop_length=hop)

In [12]:
def lib(wav):
    try:
        signal, _ = lbr.load(wav)
        sp = spec(signal)
        lsp.append(sp)
    except :
        print(wav)

In [13]:

for fol in folders:
    print(fol)
    for i in range(100):
        if i < 10:
            path = f"{pre}/{fol}/{fol}.0000{i}.wav"
        else:
            path = f"{pre}/{fol}/{fol}.000{i}.wav"
        lib(path)

blues
classical
country
disco
hiphop
jazz


/tmp/ipykernel_17/563114626.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, _ = lbr.load(wav)
/opt/conda/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/jazz/jazz.00054.wav
metal
pop
reggae
rock
